In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install sentencepiece


In [3]:
import sentencepiece as spm

In [4]:
# step 1: get the dataset
train_dataset = pd.read_csv('/kaggle/input/vietnamese-english-translation-dataset/translation_train.csv')
test_dataset = pd.read_csv('/kaggle/input/vietnamese-english-translation-dataset/translation_test.csv')
valid_dataset = pd.read_csv('/kaggle/input/vietnamese-english-translation-dataset/translation_validation.csv')

In [5]:
train_dataset.head()

,en,vi,source
0,"- Sorry, that question's not on here.","- Xin lỗi, nhưng mà ở đây không có câu hỏi đấy.",OpenSubtitles v2018
1,He wants you to come with him immediately.,Ông ấy muốn bố đi với ông ấy ngay lập tức,OpenSubtitles v2018
2,I thought we could use some company.,Tôi nghĩ chúng ta có thể muốn vài người bạn đồ...,OpenSubtitles v2018
3,It was founded in 2008 by this anonymous progr...,Nó được sáng lập vào năm 2008 bởi một lập trìn...,TED2020 v1
4,"With both of these methods, no two prints are ...","Với cả hai phương pháp, không có hai bản in nà...",TED2020 v1


In [6]:
test_dataset.head()

,en,vi,source
0,And what I think the world needs now is more c...,Và tôi nghĩ điều thế giới đang cần bây giờ là ...,TED2020 v1
1,"The group is named after Bangkok, the capital ...","Nhóm được đặt theo tên của Bangkok, thủ đô của...",wikimedia v20210402
2,It is surrounded by rivers (Simpson and Coyhai...,Nó được bao quanh bởi các con sông (Simpson và...,WikiMatrix v1
3,"Four years, and you never talked to her.","Bốn năm, và cậu chưa nói gì với cô ấy.",OpenSubtitles v2018
4,The man that's giving the test has serious dou...,Người cung cấp bài kiểm tra đó đang nghi ngờ v...,OpenSubtitles v2018


In [7]:
valid_dataset.head()

,en,vi,source
0,"In August 1764, Bertin permitted the export of...","Tháng 8 năm 1764, Bertin lại cho phép xuất khẩ...",WikiMatrix v1
1,Homeless women used to be invisible to me but ...,Tôi từng không hề để ý đến những người phụ nữ ...,TED2020 v1
2,Pumping water site for artificial infiltration...,Bơm nước cho thấm nhân tạo ở quận Sojovice.,wikimedia v20210402
3,I can't end it like this.,Tôi không thể kết thúc chuyện này như vậy.,OpenSubtitles v2018
4,"Well, I guess it's done then.","Vậy, ta đoán chúng ta đã xong.",OpenSubtitles v2018


In [8]:
# drop source column because it is redundent
train_dataset = train_dataset.drop(columns = ['source'])
test_dataset = test_dataset.drop(columns = ['source'])
valid_dataset = valid_dataset.drop(columns = ['source'])


In [9]:
train_dataset.head()

,en,vi
0,"- Sorry, that question's not on here.","- Xin lỗi, nhưng mà ở đây không có câu hỏi đấy."
1,He wants you to come with him immediately.,Ông ấy muốn bố đi với ông ấy ngay lập tức
2,I thought we could use some company.,Tôi nghĩ chúng ta có thể muốn vài người bạn đồ...
3,It was founded in 2008 by this anonymous progr...,Nó được sáng lập vào năm 2008 bởi một lập trìn...
4,"With both of these methods, no two prints are ...","Với cả hai phương pháp, không có hai bản in nà..."


In [10]:
test_dataset.head()

,en,vi
0,And what I think the world needs now is more c...,Và tôi nghĩ điều thế giới đang cần bây giờ là ...
1,"The group is named after Bangkok, the capital ...","Nhóm được đặt theo tên của Bangkok, thủ đô của..."
2,It is surrounded by rivers (Simpson and Coyhai...,Nó được bao quanh bởi các con sông (Simpson và...
3,"Four years, and you never talked to her.","Bốn năm, và cậu chưa nói gì với cô ấy."
4,The man that's giving the test has serious dou...,Người cung cấp bài kiểm tra đó đang nghi ngờ v...


In [11]:
valid_dataset.head()

,en,vi
0,"In August 1764, Bertin permitted the export of...","Tháng 8 năm 1764, Bertin lại cho phép xuất khẩ..."
1,Homeless women used to be invisible to me but ...,Tôi từng không hề để ý đến những người phụ nữ ...
2,Pumping water site for artificial infiltration...,Bơm nước cho thấm nhân tạo ở quận Sojovice.
3,I can't end it like this.,Tôi không thể kết thúc chuyện này như vậy.
4,"Well, I guess it's done then.","Vậy, ta đoán chúng ta đã xong."


In [12]:
# dictionary for all word existed 
train_dataset['type'] = 0
test_dataset['type'] = 1
valid_dataset['type'] = 2

combined_dataset = pd.concat([train_dataset, test_dataset, valid_dataset])

In [13]:
combined_dataset

,en,vi,type
0,"- Sorry, that question's not on here.","- Xin lỗi, nhưng mà ở đây không có câu hỏi đấy.",0
1,He wants you to come with him immediately.,Ông ấy muốn bố đi với ông ấy ngay lập tức,0
2,I thought we could use some company.,Tôi nghĩ chúng ta có thể muốn vài người bạn đồ...,0
3,It was founded in 2008 by this anonymous progr...,Nó được sáng lập vào năm 2008 bởi một lập trìn...,0
4,"With both of these methods, no two prints are ...","Với cả hai phương pháp, không có hai bản in nà...",0
...,...,...,...
11311,Rhozali is a professor of linguistics at the u...,Rhozali là giáo sư ngôn ngữ học tại trường đại...,2
11312,You're sorry you ever joined the Brotherhood?,Mày hối hận vì gia nhập Hội anh em à?,2
11313,"How long has it been doing that, exactly?",Em làm được như vậy bao lâu rồi?,2
11314,"I'm not going to dive into them either, but ph...","Tôi sẽ không đi sâu vào chúng, nhưng tính năng...",2


In [14]:
import re

def clean_sentence(sentence, language='English'):
    if not isinstance(sentence, str):
        return ""
    # Lowercase
    sentence = sentence.lower()

    if language == 'English':
        sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
    else:
        sentence = re.sub(r"[^a-zA-ZÀ-Ỹà-ỹĐđ\s]", "", sentence)

    # Normalize multiple spaces to a single space
    sentence = re.sub(r"\s+", " ", sentence).strip()
    return sentence


In [15]:
combined_dataset["en"] = combined_dataset["en"].apply(lambda x: clean_sentence(x, language="English"))
combined_dataset["vi"] = combined_dataset["vi"].apply(lambda x: clean_sentence(x, language="Vietnamese"))


In [16]:
len(combined_dataset)

2906992

In [17]:
combined_dataset

,en,vi,type
0,sorry that questions not on here,xin lỗi nhưng mà ở đây không có câu hỏi đấy,0
1,he wants you to come with him immediately,ông ấy muốn bố đi với ông ấy ngay lập tức,0
2,i thought we could use some company,tôi nghĩ chúng ta có thể muốn vài người bạn đồ...,0
3,it was founded in by this anonymous programmer...,nó được sáng lập vào năm bởi một lập trình viê...,0
4,with both of these methods no two prints are e...,với cả hai phương pháp không có hai bản in nào...,0
...,...,...,...
11311,rhozali is a professor of linguistics at the u...,rhozali là giáo sư ngôn ngữ học tại trường đại...,2
11312,youre sorry you ever joined the brotherhood,mày hối hận vì gia nhập hội anh em à,2
11313,how long has it been doing that exactly,em làm được như vậy bao lâu rồi,2
11314,im not going to dive into them either but phot...,tôi sẽ không đi sâu vào chúng nhưng tính năng ...,2


In [18]:
combined_dataset = combined_dataset[
    ~((combined_dataset['en'].apply(lambda x: len(x.split())) > 16) |
      (combined_dataset['vi'].apply(lambda x: len(x.split())) > 16))
]


In [19]:
len(combined_dataset)

1757875

In [20]:
combined_dataset['vi'].to_csv('train.vi', index=False, header = False)
combined_dataset['en'].to_csv('train.en', index=False, header=False)

In [21]:
import sentencepiece as spm

# Vietnamese tokenizer
spm.SentencePieceTrainer.train(
    input='train.vi',
    model_prefix='vi_tok',
    vocab_size=16000,
    character_coverage=0.9995,
    model_type='bpe',
    pad_id=0, pad_piece='<pad>',
    unk_id=1, unk_piece='<unk>',
    bos_id=2, bos_piece='<s>',
    eos_id=3, eos_piece='</s>'
)

# English tokenizer
spm.SentencePieceTrainer.train(
    input='train.en',
    model_prefix='en_tok',
    vocab_size=16000,
    character_coverage=1.0,
    model_type='bpe',
    pad_id=0, pad_piece='<pad>',
    unk_id=1, unk_piece='<unk>',
    bos_id=2, bos_piece='<s>',
    eos_id=3, eos_piece='</s>'
)


In [22]:
vi_sp = spm.SentencePieceProcessor(model_file='vi_tok.model')
en_sp = spm.SentencePieceProcessor(model_file='en_tok.model')

vietnameseWord2ID = {vi_sp.id_to_piece(i): i for i in range(vi_sp.get_piece_size())}
englishWord2ID = {en_sp.id_to_piece(i): i for i in range(en_sp.get_piece_size())}


In [23]:
vi_sp

<sentencepiece.SentencePieceProcessor; proxy of <Swig Object of type 'sentencepiece::SentencePieceProcessor *' at 0x7cc4046a1dd0> >

In [24]:
vi_sp.pad_id()

0

In [25]:
len(combined_dataset)

1757875

In [26]:
en_sp

<sentencepiece.SentencePieceProcessor; proxy of <Swig Object of type 'sentencepiece::SentencePieceProcessor *' at 0x7cc4046a1890> >

In [27]:
en_sp.pad_id()

0

In [28]:
len(englishWord2ID)

16000

In [29]:
len(vietnameseWord2ID)

16000

In [30]:
len(vietnameseWord2ID)

16000

In [31]:
englishText = combined_dataset['en'].unique()

In [32]:

longest = max(englishText, key=len)

print("Longest word:", len(longest))



Longest word: 202


In [33]:
englishText

array(['sorry that questions not on here',
       'he wants you to come with him immediately',
       'i thought we could use some company', ...,
       'rhozali is a professor of linguistics at the university of agadir specializing in oral tradition',
       'youre sorry you ever joined the brotherhood',
       'how long has it been doing that exactly'], dtype=object)

In [34]:
# import re

# def clean_sentence(sentence, language='English'):
#     if not isinstance(sentence, str):
#         return ""
#     # Lowercase
#     sentence = sentence.lower()

#     if language == 'English':
#         sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
#     else:
#         sentence = re.sub(r"[^a-zA-ZÀ-Ỹà-ỹĐđ\s]", "", sentence)

#     # Normalize multiple spaces to a single space
#     sentence = re.sub(r"\s+", " ", sentence).strip()
#     return sentence


In [35]:
vietnameseText = combined_dataset['vi'].unique()

In [36]:
vietnameseText 

array(['xin lỗi nhưng mà ở đây không có câu hỏi đấy',
       'ông ấy muốn bố đi với ông ấy ngay lập tức',
       'tôi nghĩ chúng ta có thể muốn vài người bạn đồng hành', ...,
       'rhozali là giáo sư ngôn ngữ học tại trường đại học agadir chuyên về truyền miệng',
       'mày hối hận vì gia nhập hội anh em à',
       'em làm được như vậy bao lâu rồi'], dtype=object)

In [37]:
vietnameseLongest = max(vietnameseText, key=len)
print(len(vietnameseLongest))

152


In [38]:
englishWord2ID

{'<pad>': 0,
 '<unk>': 1,
 '<s>': 2,
 '</s>': 3,
 '▁t': 4,
 'he': 5,
 '▁i': 6,
 '▁a': 7,
 '▁w': 8,
 'ou': 9,
 '▁s': 10,
 're': 11,
 '▁the': 12,
 'in': 13,
 'on': 14,
 '▁y': 15,
 '▁m': 16,
 'ha': 17,
 '▁you': 18,
 '▁c': 19,
 '▁b': 20,
 'er': 21,
 '▁d': 22,
 '▁f': 23,
 'an': 24,
 '▁o': 25,
 '▁to': 26,
 'll': 27,
 'ing': 28,
 '▁l': 29,
 '▁p': 30,
 'or': 31,
 '▁g': 32,
 'en': 33,
 '▁n': 34,
 'ed': 35,
 'is': 36,
 'es': 37,
 '▁an': 38,
 '▁in': 39,
 'hat': 40,
 'ar': 41,
 'it': 42,
 '▁th': 43,
 '▁of': 44,
 '▁he': 45,
 'at': 46,
 'om': 47,
 've': 48,
 'ow': 49,
 'as': 50,
 '▁it': 51,
 '▁h': 52,
 'ot': 53,
 '▁we': 54,
 '▁ha': 55,
 'us': 56,
 '▁and': 57,
 'et': 58,
 '▁is': 59,
 '▁be': 60,
 'le': 61,
 '▁e': 62,
 '▁that': 63,
 'ay': 64,
 'al': 65,
 '▁on': 66,
 '▁k': 67,
 'ri': 68,
 '▁re': 69,
 'gh': 70,
 'ld': 71,
 'ic': 72,
 'id': 73,
 '▁me': 74,
 '▁this': 75,
 'ver': 76,
 '▁for': 77,
 'ion': 78,
 'ut': 79,
 '▁st': 80,
 '▁was': 81,
 '▁j': 82,
 'out': 83,
 'ght': 84,
 '▁do': 85,
 'im': 86,
 'ent'

In [39]:
vietnameseWord2ID

{'<pad>': 0,
 '<unk>': 1,
 '<s>': 2,
 '</s>': 3,
 '▁t': 4,
 '▁c': 5,
 'ng': 6,
 '▁đ': 7,
 '▁n': 8,
 '▁l': 9,
 '▁th': 10,
 '▁ch': 11,
 '▁v': 12,
 '▁m': 13,
 '▁b': 14,
 '▁k': 15,
 '▁h': 16,
 'nh': 17,
 '▁s': 18,
 '▁kh': 19,
 '▁nh': 20,
 '▁g': 21,
 'ông': 22,
 '▁là': 23,
 '▁ng': 24,
 'ôi': 25,
 '▁tr': 26,
 '▁tôi': 27,
 '▁p': 28,
 '▁không': 29,
 '▁a': 30,
 '▁r': 31,
 '▁ph': 32,
 '▁có': 33,
 '▁d': 34,
 '▁gi': 35,
 '▁và': 36,
 '▁anh': 37,
 '▁ta': 38,
 'ên': 39,
 'iế': 40,
 'ời': 41,
 '▁đư': 42,
 'ủa': 43,
 '▁của': 44,
 '▁nó': 45,
 'ày': 46,
 'ột': 47,
 'ới': 48,
 '▁chú': 49,
 '▁một': 50,
 'ác': 51,
 'ấy': 52,
 'qu': 53,
 '▁đi': 54,
 'ợc': 55,
 '▁qu': 56,
 '▁đó': 57,
 '▁được': 58,
 '▁x': 59,
 '▁đã': 60,
 'ất': 61,
 '▁ngư': 62,
 '▁chúng': 63,
 'iệ': 64,
 'on': 65,
 'uy': 66,
 'ại': 67,
 'ải': 68,
 '▁đâ': 69,
 '▁sẽ': 70,
 '▁người': 71,
 '▁như': 72,
 'an': 73,
 '▁này': 74,
 '▁ngh': 75,
 '▁sa': 76,
 '▁cho': 77,
 '▁phải': 78,
 '▁cô': 79,
 'em': 80,
 '▁làm': 81,
 'ong': 82,
 '▁ở': 83,
 'uố': 84,
 '

In [40]:
input_vocab_size = len(englishWord2ID)
target_vocab_size = len(vietnameseWord2ID)

In [41]:
input_vocab_size, target_vocab_size

(16000, 16000)

In [42]:
import torch
import torch.nn as nn

In [43]:
# build some function need for transformer by sinusoidal 
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, seq_len):
        super().__init__()  
        pe = torch.zeros(seq_len, embed_dim)
        position = torch.arange(0, seq_len, dtype = torch.float).unsqueeze(1) # [max_len, 1]
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * -(math.log(10000.0) / embed_dim)) # (1/ 10000) ^ (2i / d_model)
        pe[:, 0 :: 2] = torch.sin(position * div_term) # the even pos
        pe[:, 1:: 2] = torch.cos(position * div_term) # the odd pos
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        seq_len = x.size(1)
        x = x + self.pe[:, :seq_len, :]
        return x

In [44]:
class MultiheadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0, "Embedding dimension must be divisible by number of heads"
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.heads_dim = self.embed_dim // self.num_heads
        self.W_q = nn.Linear(embed_dim, embed_dim, bias = False)
        self.W_k = nn.Linear(embed_dim, embed_dim, bias = False)
        self.W_v = nn.Linear(embed_dim, embed_dim, bias = False)
        self.W_o = nn.Linear(embed_dim, embed_dim, bias = False)
        self.dropout = nn.Dropout(0.3)
    def forward(self, x, mask=None, x_cross = None):
        # x = the input 
        # x_cross = x from encoder part
        batch_size, tgt_seq_length, embed_dim = x.shape
        Q = self.W_q(x) # [batch, tgt_target_length, embed_dim]
        if x_cross is None:
            K = self.W_k(x) # [batch, tgt_input_length, embed_dim]
            V = self.W_v(x) # [batch, tgt_input_length, embed_dim]
            src_seq_length = tgt_seq_length
        else:
            K = self.W_k(x_cross) # [batch, src_input_length, embed_dim]
            V = self.W_v(x_cross) # [batch, src_input_length, embed_dim]
            src_seq_length = x_cross.shape[1]
        Q = Q.view(batch_size, tgt_seq_length, self.num_heads, self.heads_dim).transpose(1, 2) # [batch. num_heads, tgt_seq_length, heads_dim]
        K = K.view(batch_size, src_seq_length, self.num_heads, self.heads_dim).transpose(1, 2) # [batch, num_heads, src_seq_length, heads_dim]
        V = V.view(batch_size, src_seq_length, self.num_heads, self.heads_dim).transpose(1, 2) # [batch, num_heads, src_seq_length, heads_dim]
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.heads_dim ** 0.5) # attention score = Q * transpose(T) / sqrt(dim_k) # list of score
        # scores = [batch, num_heads, tgt_seq_length, src_seq_length]
        # get max 
        if mask is not None:
            scores = scores + mask
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        # attention(Q, K, V)
        attended_value = torch.matmul(attention_weights, V) # [batch, num_heads, tgt_seq_length, heads_dim]
        # multihead attention 
        attended_value = attended_value.transpose(1, 2).contiguous().view(batch_size, tgt_seq_length, embed_dim)
        output = self.W_o(attended_value)
        return output # [batch, tgt_seq_len, embed_dim]

In [45]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, embed_dim, hidden_dim):
        '''
        Positional-wise FF Networks
        # formula FFN(x) = max(0, xW1 + B1)*W2 + b2
        # 1 linear -> 1 relu -> 1 linear
        embed_dim = d_model = dimension of model
        hidden_dim = dimension of FFN layer
        '''
        super().__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, embed_dim)
        # for potential speed up
        # pre-normalize the weights for stabling training
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
    def forward(self, prev_input):
        batch_size, seq_length, embed_dim = prev_input.size()
        f1 = F.relu(self.fc1(prev_input))
        f2 = self.fc2(f1)
        return f2 

In [46]:
class TransformerEncoder(nn.Module):
    def __init__(self,embed_dim, num_heads, hidden_dim, dropout):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.hidden_dim = hidden_dim
        self.att = MultiheadAttention(embed_dim=self.embed_dim, num_heads = self.num_heads)
        self.ffn = PositionWiseFeedForward(embed_dim = self.embed_dim, hidden_dim = self.hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.LayerNorm_att = nn.LayerNorm(self.embed_dim)
        self.LayerNorm_ffn = nn.LayerNorm(self.embed_dim)
    def forward(self, embed_input, padding_mask = None):
        batch_size, seq_len, _ = embed_input.size()
        att_sublayer = self.att(x=embed_input)
        att_sublayer = self.dropout(att_sublayer)
        att_sublayer = self.LayerNorm_att(embed_input + att_sublayer)
        ffn_sublayer = self.ffn(att_sublayer)
        ffn_sublayer = self.dropout(ffn_sublayer)
        ffn_sublayer = self.LayerNorm_ffn(ffn_sublayer + att_sublayer)
        return ffn_sublayer        

In [47]:
def generate_subsequent_mask(target_seq_length):
    """
    Create a causal mask (upper triangular matrix filled with 0s and 1s).
    1 = keep, 0 = mask out
    """    
    mask = torch.triu(torch.ones(target_seq_length, target_seq_length), diagonal=1)
    return mask.masked_fill(mask == 1, float('-inf'))


In [48]:
# Create padding mask (1 for real tokens, 0 for padding)
def create_pad_mask(seq, pad_id=en_sp.pad_id()):
    return (seq != pad_id).unsqueeze(1).unsqueeze(2)  # [batch, 1, 1, seq_len]


In [49]:
class TransformerDecoder(nn.Module):
    def __init__(self, embed_dim, num_heads, hidden_dim, dropout):
        super().__init__()  
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.hidden_dim = hidden_dim
        self.masked_att = MultiheadAttention(embed_dim = self.embed_dim, num_heads = self.num_heads)
        self.att = MultiheadAttention(embed_dim = self.embed_dim, num_heads = self.num_heads)
        self.ffn = PositionWiseFeedForward(embed_dim = self.embed_dim, hidden_dim = self.hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.LayerNorm_maskedatt = nn.LayerNorm(self.embed_dim)
        self.LayerNorm_att = nn.LayerNorm(self.embed_dim)
        self.LayerNorm_ffn = nn.LayerNorm(self.embed_dim)
    def forward(self, embed_input, encoder_input):
        batch_size, tgt_seq_length, _ = embed_input.size()
        _, src_seq_length, _ = encoder_input.size()
        mask = generate_subsequent_mask(tgt_seq_length).to(embed_input.device)
        mask = mask.unsqueeze(0).unsqueeze(0) # [batch, num_heads, tgt_len, tgt_len]
        maskedatt_sublayer = self.masked_att(x=embed_input, mask=mask)
        maskedatt_sublayer = self.dropout(maskedatt_sublayer)
        maskedatt_sublayer = self.LayerNorm_maskedatt(maskedatt_sublayer + embed_input)
        att_sublayer = self.att(x=maskedatt_sublayer, x_cross = encoder_input)
        att_sublayer = self.dropout(att_sublayer)
        att_sublayer = self.LayerNorm_att(att_sublayer + maskedatt_sublayer)
        ffn_sublayer = self.ffn(att_sublayer)
        ffn_sublayer = self.dropout(ffn_sublayer)
        ffn_sublayer = self.LayerNorm_ffn(ffn_sublayer + att_sublayer)
        return ffn_sublayer # [batch, tgt_sequence_length, embed_dim]

In [50]:
class TransformerEncoderDecoder(nn.Module):
    def __init__(self, embed_dim, num_heads, hidden_dim, dropout, num_layers = 6):
        super().__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.encoder = nn.ModuleList([TransformerEncoder(embed_dim = self.embed_dim, num_heads = self.num_heads, hidden_dim = self.hidden_dim, dropout = dropout) for _ in range(num_layers)])
        self.decoder = nn.ModuleList([TransformerDecoder(embed_dim = self.embed_dim, num_heads = self.num_heads, hidden_dim = self.hidden_dim, dropout = dropout) for _ in range(num_layers)])
    def forward(self, embed_encoder_input, embed_decoder_input):
        encoder_output = embed_encoder_input
        for layer in self.encoder:
            encoder_output = layer(encoder_output)
    
        decoder_output = embed_decoder_input
        for layer in self.decoder:
            decoder_output = layer(decoder_output, encoder_output)
    
        return decoder_output

In [51]:
class EmbeddingForTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim, seq_len):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.positionalEncoding = PositionalEncoding(embed_dim = embed_dim, seq_len = seq_len)
    def forward(self, x):
        embed = self.embedding(x) * math.sqrt(self.embedding.embedding_dim) # for normalization
        embed = self.positionalEncoding(embed)
        return embed

In [52]:
import torch.nn.functional as F

class TransformerArchitecture(nn.Module):
    def __init__(self, num_layers, embed_dim, hidden_dim, num_heads, src_vocab_size, tgt_vocab_size,
                 src_seq_len, tgt_seq_len, max_position_embeddings=512, dropout=0.1):
        super().__init__()
        self.tgt_vocab_size = tgt_vocab_size
        self.src_vocab_size = src_vocab_size
        
        self.encoder_embedding = EmbeddingForTransformer(vocab_size=src_vocab_size, embed_dim=embed_dim, seq_len=src_seq_len)
        self.decoder_embedding = EmbeddingForTransformer(vocab_size=tgt_vocab_size, embed_dim=embed_dim, seq_len=tgt_seq_len)
        
        self.encoder_decoder = TransformerEncoderDecoder(embed_dim, num_heads, hidden_dim, dropout, num_layers)
        
        # Output layer
        self.projection = nn.Linear(embed_dim, tgt_vocab_size)

    def shift_target_right(self, tgt_tokens):
        batch_size, seq_len = tgt_tokens.size()
        start_tokens = torch.full(
            (batch_size, 1),
            vietnameseWord2ID['<s>'],
            dtype=tgt_tokens.dtype,
            device=tgt_tokens.device  # <-- move to same device
        )
        shifted_tokens = torch.cat([start_tokens, tgt_tokens[:, :-1]], dim=1)
        return shifted_tokens

    def forward(self, src_tokens, tgt_tokens):
        shifted_tgt_tokens = self.shift_target_right(tgt_tokens)
        embed_input = self.encoder_embedding(src_tokens)
        embed_output = self.decoder_embedding(shifted_tgt_tokens)
        decoder_output = self.encoder_decoder(embed_encoder_input=embed_input, embed_decoder_input=embed_output)
        logits = self.projection(decoder_output)
        # log_probs = F.log_softmax(logits, dim=-1)
        return logits


In [53]:
# max seq_length english = 6963
# max_seq_length_vietnamese = 7188
# divide back to the train, test and valid dataset
train_dataset = combined_dataset[combined_dataset['type'] == 0]
test_dataset = combined_dataset[combined_dataset['type'] == 1]
valid_dataset = combined_dataset[combined_dataset['type'] == 2]

In [54]:
train_dataset = train_dataset.drop(columns = ['type'])
test_dataset = test_dataset.drop(columns = ['type'])
valid_dataset = valid_dataset.drop(columns = ['type'])


In [55]:
train_dataset.head()

,en,vi
0,sorry that questions not on here,xin lỗi nhưng mà ở đây không có câu hỏi đấy
1,he wants you to come with him immediately,ông ấy muốn bố đi với ông ấy ngay lập tức
2,i thought we could use some company,tôi nghĩ chúng ta có thể muốn vài người bạn đồ...
6,i have lived to see something which i never ex...,ta đã sống để thấy điều ta không bao giờ mong đợi
8,welcome him as your brother,chào mừng nó như anh em của các con


In [56]:
from torch.utils.data import Dataset, DataLoader

In [57]:
def encode_sentence_sp(sentence, max_len, sp_model):
    output = [sp_model.bos_id()]     
    piece_ids = sp_model.encode(sentence, out_type=int)
    output.extend(piece_ids)
    output.append(sp_model.eos_id())  # </s> or <EOS>
    
    # Pad if needed
    if len(output) < max_len:
        output.extend([sp_model.pad_id()] * (max_len - len(output)))
    else:
        output = output[:max_len]  
    
    return output


In [58]:
import sentencepiece as spm

# Load your .model
sp = spm.SentencePieceProcessor()
sp.load("vi_tok.model")  # or "en_tok.model" for English

# Encode sentence
encoded = encode_sentence_sp("tôi là sinh viên", max_len=20, sp_model=sp)

print(encoded)  # List of subword IDs
decoded = sp.decode([id for id in encoded if id > 0 and id < sp.get_piece_size()])
print(decoded)

[2, 27, 23, 357, 441, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tôi là sinh viên


In [59]:
vietnameseModel = spm.SentencePieceProcessor()
vietnameseModel.load('/kaggle/working/vi_tok.model')
englishModel = spm.SentencePieceProcessor()
englishModel.load('/kaggle/working/en_tok.model')

True

In [60]:
train_english_sentence = train_dataset['en'].tolist()
train_vietnamese_sentence = train_dataset['vi'].tolist()
test_english_sentence = test_dataset['en'].tolist()
test_vietnamese_sentence = test_dataset['vi'].tolist()
valid_english_sentence = valid_dataset['en'].tolist()
valid_vietnamese_sentence = valid_dataset['vi'].tolist()


In [61]:
# Count how long each sentence is (in tokens/words)
english_lengths = [len(sen.split()) for sen in train_english_sentence]
vietnamese_lengths = [len(sen.split()) for sen in train_vietnamese_sentence]

# Show the actual longest length
print("Max English Sentence Length:", max(english_lengths))
print("Max Vietnamese Sentence Length:", max(vietnamese_lengths))

# Optionally: find the longest sentence
max_en_len = max(english_lengths)
max_vi_len = max(vietnamese_lengths)

longest_english = train_english_sentence[english_lengths.index(max_en_len)]
longest_vietnamese = train_vietnamese_sentence[vietnamese_lengths.index(max_vi_len)]

print("\nLongest English Sentence Sample:")
print(longest_english)

print("\nLongest Vietnamese Sentence Sample:")
print(longest_vietnamese)


Max English Sentence Length: 16
Max Vietnamese Sentence Length: 16

Longest English Sentence Sample:
ashurbanipal celebrated a service of reconciliation but did not venture to take the hands of bel

Longest Vietnamese Sentence Sample:
tôi đã uống vài viên thuốc lúc nãy giờ tôi phải giải thích cho các bạn


In [62]:
# Count how long each sentence is (in tokens/words)
english_lengths = [len(sen.split()) for sen in valid_english_sentence]
vietnamese_lengths = [len(sen.split()) for sen in valid_vietnamese_sentence]

# Show the actual longest length
print("Max English Sentence Length:", max(english_lengths))
print("Max Vietnamese Sentence Length:", max(vietnamese_lengths))

# Optionally: find the longest sentence
max_en_len = max(english_lengths)
max_vi_len = max(vietnamese_lengths)

longest_english = valid_english_sentence[english_lengths.index(max_en_len)]
longest_vietnamese = valid_vietnamese_sentence[vietnamese_lengths.index(max_vi_len)]

print("\nLongest English Sentence Sample:")
print(longest_english)

print("\nLongest Vietnamese Sentence Sample:")
print(longest_vietnamese)


Max English Sentence Length: 16
Max Vietnamese Sentence Length: 16

Longest English Sentence Sample:
before the end of she had departed san diego california and arrived in charleston south carolina

Longest Vietnamese Sentence Sample:
ở đây chúng ta có qatar hiện tại và ở kia ta có bangladesh hiện tại


In [63]:
import numpy as np

x_train = np.array([encode_sentence_sp(sen, 20, englishModel) for sen in train_english_sentence], dtype=np.int32)
y_train = np.array([encode_sentence_sp(sen, 20, vietnameseModel) for sen in train_vietnamese_sentence], dtype=np.int32)

x_test = np.array([encode_sentence_sp(sen, 20,englishModel) for sen in test_english_sentence], dtype=np.int32)
y_test = np.array([encode_sentence_sp(sen, 20, vietnameseModel) for sen in test_vietnamese_sentence], dtype=np.int32)

x_valid = np.array([encode_sentence_sp(sen, 20, englishModel) for sen in valid_english_sentence], dtype=np.int32)
y_valid = np.array([encode_sentence_sp(sen, 20, vietnameseModel) for sen in valid_vietnamese_sentence], dtype=np.int32)


In [64]:
# x_train[:3]

In [65]:
# creating dataset 
class TranslationDataset(Dataset):
    def __init__(self, original_text, translated_text):
        self.original_text = original_text
        self.translated_text = translated_text
    def __len__(self):
        return len(self.original_text)
    def __getitem__(self, idx):
        return torch.tensor(self.original_text[idx]), torch.tensor(self.translated_text[idx])

In [66]:
new_train_dataset = TranslationDataset(x_train, y_train)
new_test_dataset = TranslationDataset(x_test, y_test)
new_valid_dataset = TranslationDataset(x_valid, y_valid)

In [67]:
train_dataloader = DataLoader(new_train_dataset, batch_size = 512, shuffle = False)
test_dataloader = DataLoader(new_test_dataset, batch_size = 512, shuffle = False)
valid_dataloader = DataLoader(new_valid_dataset, batch_size = 512, shuffle = False)

In [68]:
train_dataloader

In [69]:
import math

In [70]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [71]:
device  = 'cuda' if torch.cuda.is_available() else "cpu"

In [72]:
def evaluate(eval_dataloader, cur_status='eval'):
    criterion = nn.CrossEntropyLoss(ignore_index=vietnameseModel.pad_id(), label_smoothing = 0.1)
    model.eval()
    total_loss = 0.0
    for batch_idx, (src_batch, tgt_batch) in enumerate(eval_dataloader):
        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)
        output = model(src_batch, tgt_batch)
        output = output[:, :-1, :]  # shift output
        target = tgt_batch[:, 1:]      # shift target
        loss = criterion(output.reshape(-1, target_vocab_size), target.reshape(-1).long())
        total_loss += loss.item()
    if cur_status == 'train':
        model.train()
    return total_loss / len(eval_dataloader)


In [73]:
import time

In [74]:
# input_vocab_size, target_vocab_size
model = TransformerArchitecture(num_layers = 6, embed_dim = 512, hidden_dim = 1024, num_heads = 8, src_vocab_size = input_vocab_size, tgt_vocab_size = target_vocab_size,
                 src_seq_len=20, tgt_seq_len=20, max_position_embeddings=20, dropout=0.1)
model = model.to(device)
# criterion = nn.NLLLoss(ignore_index=vietnameseModel.pad_id())
criterion = nn.CrossEntropyLoss(ignore_index=vietnameseModel.pad_id(), label_smoothing = 0.1)
print("Number of parameters: ", sum(p.numel() for p in model.parameters() if p.requires_grad))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# def noam_schedule(step):
#     d_model = 512
#     warmup_steps = 4000
#     if step == 0:
#         step = 1  # prevent division by zero
#     return (d_model ** -0.5) * min(step ** -0.5, step * (warmup_steps ** -1.5))


# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=noam_schedule)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    startTime = time.time()
    total_loss = 0
    print(f"Starting epoch {epoch + 1}")
    for batch_idx, (src_batch, tgt_batch) in enumerate(train_dataloader):
        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)
        optimizer.zero_grad()
        output = model(src_batch, tgt_batch)
        output = output[:, :-1, :]  # shift output
        target = tgt_batch[:, 1:]      # shift target
        loss = criterion(output.reshape(-1, target_vocab_size), target.reshape(-1).long())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        # scheduler.step()
        total_loss += loss.item()
        if (batch_idx + 1) % 150 == 0:
            print(f"Batch {batch_idx+1}: Loss = {loss.item():.4f}")
    eval_loss = evaluate(valid_dataloader, cur_status ='train')
    endTime = time.time()
    print(f"Time consumption: {(endTime - startTime) / 60.0} minutes")
    print(f"Epoch {epoch+1}: loss = {total_loss/len(train_dataloader):.4f}")
    print(f"Validation loss: ", eval_loss )


Number of parameters:  56098432
Starting epoch 1
Batch 150: Loss = 6.6958
Batch 300: Loss = 6.4671
Batch 450: Loss = 6.1565
Batch 600: Loss = 6.2288
Batch 750: Loss = 6.0897
Batch 900: Loss = 6.0225
Batch 1050: Loss = 5.9093
Batch 1200: Loss = 5.8735
Batch 1350: Loss = 5.8585
Batch 1500: Loss = 5.8120
Batch 1650: Loss = 5.7400
Batch 1800: Loss = 5.7203
Batch 1950: Loss = 5.6876
Batch 2100: Loss = 5.6564
Batch 2250: Loss = 5.6400
Batch 2400: Loss = 5.5997
Batch 2550: Loss = 5.5995
Batch 2700: Loss = 5.4353
Batch 2850: Loss = 5.4637
Batch 3000: Loss = 5.4260
Batch 3150: Loss = 5.4070
Batch 3300: Loss = 5.2928
Time consumption: 27.290338309605918 minutes
Epoch 1: loss = 5.8496
Validation loss:  5.063202491173377
Starting epoch 2
Batch 150: Loss = 5.3528
Batch 300: Loss = 5.3137
Batch 450: Loss = 5.0868
Batch 600: Loss = 5.2847
Batch 750: Loss = 5.1967
Batch 900: Loss = 5.1886
Batch 1050: Loss = 5.1463
Batch 1200: Loss = 5.0956
Batch 1350: Loss = 5.1065
Batch 1500: Loss = 5.1288
Batch 1650

In [75]:
model.eval()
text = "how are you today"
encoded_text = encode_sentence_sp(text, 20, englishModel)
src_tensor = torch.tensor(encoded_text, dtype=torch.long).unsqueeze(0).to(device)

max_len = 20
translated_indices = [vietnameseWord2ID['<s>']]

for _ in range(max_len - 1):
    tgt_tensor = torch.tensor(translated_indices, dtype=torch.long).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(src_tensor, tgt_tensor)
    
    next_token_logits = output[0, -1, :]
    next_token = next_token_logits.argmax(dim=-1).item()
    
    translated_indices.append(next_token)
    
    if next_token == vietnameseWord2ID['</s>']:
        break


In [76]:
translated_indices

[2, 590, 381, 37, 1358, 159, 3]

In [77]:
decoded = sp.decode([id for id in translated_indices if id > 0 and id < sp.get_piece_size()])


In [78]:
decoded

'hôm nay anh khỏe nào'

In [79]:
evaluate(test_dataloader)

3.770802846321693

In [80]:
def generateTranslation(sentence):
    # preprocess the sentence first for suit with training model
    sentence = clean_sentence(sentence, language = 'English')
    encoded_text = encode_sentence_sp(sentence, 20, englishModel)
    src_tensor = torch.tensor(encoded_text, dtype=torch.long).unsqueeze(0).to(device) # batch_size, seq_len
    max_len = 20
    translated_indices = [vietnameseWord2ID['<s>']] # start by the start sequence part
    for _ in range(max_len - 1):
        tgt_tensor = torch.tensor(translated_indices, dtype=torch.long).unsqueeze(0).to(device) # add batch_size dim
        with torch.no_grad():
            output = model(src_tensor, tgt_tensor)
        next_token_logits = output[0, -1, :]
        next_token = next_token_logits.argmax(dim=-1).item()
        translated_indices.append(next_token)
        if next_token == vietnameseWord2ID['</s>']:
            break
    decoded = vietnameseModel.decode([id for id in translated_indices if id > 0 and id < sp.get_piece_size()])
    return decoded


In [81]:
# # generate for some test:
# for batch_idx, (src_batch, tgt_batch) in enumerate(eval_dataloader):
    

In [82]:
generateTranslation("i will have a good day")

'tôi sẽ có một ngày tốt lành'

In [83]:
for batch_idx, (src_batch, tgt_batch) in enumerate(test_dataloader):
    if batch_idx >= 3:
        break  # Stop after 3 batches

    src_batch = src_batch.to(device)
    tgt_batch = tgt_batch.to(device)

    src_batch = src_batch[:4]
    tgt_batch = tgt_batch[:4]

    for src in src_batch:
        filtered_ids = [id.item() for id in src if 0 < id < englishModel.get_piece_size()]
        print("Token IDs:", filtered_ids)

        # Decode to original sentence
        original_sentence = englishModel.decode(filtered_ids)
        print("Original:", original_sentence)

        # Generate translation
        translated_sentence = generateTranslation(original_sentence)
        print("Translated:", translated_sentence)


Token IDs: [2, 57, 97, 6, 196, 12, 506, 1435, 186, 59, 301, 8175, 3]
Original: and what i think the world needs now is more connections
Translated: và điều tôi nghĩ thế giới cần cần giờ giờ nối nối hơn hơn
Token IDs: [2, 51, 59, 5664, 235, 5607, 850, 7948, 57, 13545, 17, 4687, 57, 235, 3651, 3]
Original: it is surrounded by rivers simpson and coyhaique and by mountains
Translated: nó được bao quanh bởi các sông sông và và sói sói bởi núi
Token IDs: [2, 782, 447, 57, 18, 375, 2576, 26, 201, 3]
Original: four years and you never talked to her
Translated: bốn năm và anh chưa bao giờ nói chuyện với cô ấy
Token IDs: [2, 5649, 12, 13156, 3417, 3]
Original: mmm the constitutional lawyer
Translated: luật luật pháp pháp
Token IDs: [2, 18, 391, 45, 81, 7, 250, 491, 1213, 7800, 3]
Original: you said he was a chubby loser
Translated: anh nói hắn ta kẻ kẻ bại bại
Token IDs: [2, 97, 159, 75, 4115, 3]
Original: what about this bottle
Translated: còn cái chai này thì sao
Token IDs: [2, 1747, 109, 411,